In [1]:
import pandas as pd
import nltk
from transformers import pipeline, set_seed, AutoTokenizer, AutoModelForSeq2SeqLM
import matplotlib.pyplot as plt
from datasets import load_dataset, load_from_disk, load_metric
from nltk.tokenize import sent_tokenize
from tqdm import tqdm


DATASET = "../artifacts/data_ingestion/samsum_dataset" 

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
# tokenizer = AutoTokenizer.from_pretrained("google/pegasus-large")
# model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-large")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
dataset = load_from_disk(DATASET)

In [4]:
def vectorizer(batch):
 
  input = tokenizer(batch['dialogue'], max_length = 1024, truncation = True)
  with tokenizer.as_target_tokenizer():
    target = tokenizer(batch['summary'], max_length = 128, truncation = True)
  return {
      'input_ids': input['input_ids'],
      'attention_mask': input['attention_mask'],
      'labels': target['input_ids']
  }

In [5]:
dataset_pt = dataset.map(vectorizer, batched = True)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]c:\Users\msova\anaconda3\envs\summaraize\Lib\site-packages\transformers\tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 819/819 [00:00<00:00, 2708.90 examples/s]


In [ ]:
model = Sequential()
model.add.layers.LSTM(512)
model.add.layers.LSTM(512)

In [6]:
# Training
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer

data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)
training_arg = TrainingArguments(
    output_dir = 'bart-samsum', num_train_epochs = 10, warmup_steps = 500,
    per_device_train_batch_size = 4, per_device_eval_batch_size = 4, weight_decay = 0.01,
    logging_steps = 10, eval_strategy = 'steps', eval_steps = 500, save_steps = 1e6,
    gradient_accumulation_steps = 16
    )



In [7]:
trainer = Trainer (
    model,
    training_arg,
    data_collator = data_collator,
    train_dataset = dataset_pt['train'],
    eval_dataset = dataset_pt['validation']
    )


In [8]:
trainer.train()

  0%|          | 0/2300 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 10.59 GiB is allocated by PyTorch, and 166.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)